In [134]:
# Imports
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import random
import numpy as np
import pandas as pd
from factor_analyzer import FactorAnalyzer
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn import metrics
import random
from sklearn.datasets import load_boston
from scipy.sparse import csr_matrix

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix

In [135]:
# import the dataset
import pandas as pd
df=pd.read_csv('movie_dataset_ml')

C:\Users\Fatoo\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (12,13) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [136]:
df['movie_id']= df['tconst'].str.split('tt', 1).str[1]
df

,Unnamed: 0,year,month,day,user_id_maped,program_name,duration_seconds,program_class,program_genre,hd,title,tconst,seasonNumber,episodeNumber,averageRating,numVotes,ordering,region,isOriginalTitle,movie_id
0,0,2018,1,27,11660,Kidnap,85,MOVIE,Action,1,Kidnap,tt0534484,3,18,6.9,62,1,\N,1,0534484
1,1,2018,1,27,11660,Kidnap,85,MOVIE,Action,1,Kidnap,tt0534484,3,18,6.9,62,2,US,0,0534484
2,2,2018,1,27,11660,Kidnap,85,MOVIE,Action,1,Kidnap,tt0649245,7,11,8.0,93,1,US,0,0649245
3,3,2018,1,27,11660,Kidnap,85,MOVIE,Action,1,Kidnap,tt0673627,2,8,7.6,65,1,US,0,0673627
4,4,2018,1,27,11660,Kidnap,85,MOVIE,Action,1,Kidnap,tt0684236,2,4,5.8,34,1,US,0,0684236
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
354634,354634,2018,4,20,32570,Dreamcatcher,79,MOVIE,Drama,0,Dreamcatcher,tt3549296,2,15,8.7,755,1,US,0,3549296
354635,354635,2018,4,20,32570,Dreamcatcher,79,MOVIE,Drama,0,Dreamcatcher,tt4799922,5,5,8.3,1182,1,US,0,4799922
354636,354636,2018,4,16,22557,Dreamcatcher,16,MOVIE,Drama,0,Dreamcatcher,tt11131848,3,11,8.8,14,1,US,0,11131848
354637,354637,2018,4,16,22557,Dreamcatcher,16,MOVIE,Drama,0,Dreamcatcher,tt3549296,2,15,8.7,755,1,US,0,3549296


In [137]:
#df= df.drop_duplicates(subset=['tconst'] )
#df

In [138]:
df['movie_id']=pd.to_numeric(df['movie_id'])

In [140]:
# Data Processing
# Converting Genres into different columns 
# Here we just create columns and put there initial value as 0
x = df.program_genre
a = list()
for i in x:
    abc = i
    a.append(abc.split('|'))
a = pd.DataFrame(a)   
b = a[0].unique()
for i in b:
    df[i] = 0

In [141]:
df.sample(5)

,Unnamed: 0,year,month,day,user_id_maped,program_name,duration_seconds,program_class,program_genre,hd,...,Horror,Drama,Animation,NOT_DEFINED_IN_UMS,Crime,Thriller,Biography,Family,Adventure,Documentary
279635,279635,2018,2,7,29680,The Boy Next Door,13,MOVIE,Thriller,1,...,0,0,0,0,0,0,0,0,0,0
241913,241913,2018,4,11,16872,Cinderella,64,MOVIE,Animation,0,...,0,0,0,0,0,0,0,0,0,0
21231,21231,2017,12,17,8291,Alien: Covenant,89,MOVIE,Horror,1,...,0,0,0,0,0,0,0,0,0,0
84559,84559,2017,11,13,8394,Gifted,109,MOVIE,Drama,1,...,0,0,0,0,0,0,0,0,0,0
337794,337794,2017,12,15,30166,The Equalizer,114,MOVIE,Action,0,...,0,0,0,0,0,0,0,0,0,0


In [142]:
# we assign 1 to all the columns which are present in the Genres
for i in b:
    df.loc[df['program_genre'].str.contains(i), i] = 1

In [143]:
df.sample(2)

,Unnamed: 0,year,month,day,user_id_maped,program_name,duration_seconds,program_class,program_genre,hd,...,Horror,Drama,Animation,NOT_DEFINED_IN_UMS,Crime,Thriller,Biography,Family,Adventure,Documentary
115772,115772,2017,9,22,9176,Brimstone,98,MOVIE,Thriller,1,...,0,0,0,0,0,1,0,0,0,0
213891,213891,2017,9,4,32300,24 :00 AM - :00 AM,25,SERIES/EPISODES,Drama,0,...,0,1,0,0,0,0,0,0,0,0


In [144]:
# Now there is no use of genre 
data = df.drop(['Unnamed: 0','year','month','duration_seconds','day','title','region','ordering',
                'seasonNumber','numVotes','episodeNumber','program_name','hd','tconst','isOriginalTitle','program_class','program_genre'],axis =1)
data

,user_id_maped,averageRating,movie_id,Action,Comedy,Horror,Drama,Animation,NOT_DEFINED_IN_UMS,Crime,Thriller,Biography,Family,Adventure,Documentary
0,11660,6.9,534484,1,0,0,0,0,0,0,0,0,0,0,0
1,11660,6.9,534484,1,0,0,0,0,0,0,0,0,0,0,0
2,11660,8.0,649245,1,0,0,0,0,0,0,0,0,0,0,0
3,11660,7.6,673627,1,0,0,0,0,0,0,0,0,0,0,0
4,11660,5.8,684236,1,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
354634,32570,8.7,3549296,0,0,0,1,0,0,0,0,0,0,0,0
354635,32570,8.3,4799922,0,0,0,1,0,0,0,0,0,0,0,0
354636,22557,8.8,11131848,0,0,0,1,0,0,0,0,0,0,0,0
354637,22557,8.7,3549296,0,0,0,1,0,0,0,0,0,0,0,0


In [145]:
kmeanModel = KMeans(n_clusters=8)
kmeanModel.fit(data)

KMeans()

In [146]:
# Creating an extra column in data for storing the cluster values
data['Cluster'] = kmeanModel.labels_
data['Cluster'].sample(n=10)

81159     0
90229     2
925       0
215402    0
213746    0
74299     5
299432    0
123189    0
176394    0
145006    5
Name: Cluster, dtype: int32

In [147]:
data['Cluster'].value_counts()

0    163261
6     62372
5     28522
2     27194
1     23933
4     20949
7     16110
3     12298
Name: Cluster, dtype: int64

In [148]:
data.head()

,user_id_maped,averageRating,movie_id,Action,Comedy,Horror,Drama,Animation,NOT_DEFINED_IN_UMS,Crime,Thriller,Biography,Family,Adventure,Documentary,Cluster
0,11660,6.9,534484,1,0,0,0,0,0,0,0,0,0,0,0,0
1,11660,6.9,534484,1,0,0,0,0,0,0,0,0,0,0,0,0
2,11660,8.0,649245,1,0,0,0,0,0,0,0,0,0,0,0,0
3,11660,7.6,673627,1,0,0,0,0,0,0,0,0,0,0,0,0
4,11660,5.8,684236,1,0,0,0,0,0,0,0,0,0,0,0,0


In [202]:
#list of clusters and clusters
e = []
def movie_c(group):
    a = pd.DataFrame(group)
    #b = values count of clusters 
    b = pd.DataFrame(a['Cluster'].value_counts())
    #movie_id index in d
    d = a.index 
    c = [a['movie_id'][d[0]],int(b.idxmax())]
    e.append(c) 

In [203]:
#Call the function and give it th data grouped by movie id
data.groupby("movie_id").apply(lambda x: movie_c(x))

""


In [201]:
#Convert the list to dataframe
e = pd.DataFrame(e)

In [183]:
e.head()

,0,1
0,57010,0
1,67905,0
2,74301,0
3,74690,0
4,75671,0


In [184]:
e.rename(columns = {0:'movie_id',1:'Cluster'},inplace=True)

In [185]:
e.sample(10)

,movie_id,Cluster
383,658648,0
1267,11417062,7
843,2104811,6
798,1686421,6
396,668622,0
582,836866,0
391,665738,0
363,653503,0
1250,10912540,7
254,598091,0


In [189]:
# import the dataset
import pandas as pd
df1=pd.read_csv('movie_dataset_ml')
df1['movie_id']= df1['tconst'].str.split('tt', 1).str[1]
df1['movie_id']=pd.to_numeric(df1['movie_id'])

In [190]:
df1=df1[['title','movie_id']]

In [191]:
new_data = pd.merge(e , df1)

In [192]:
new_data.sample()

,movie_id,Cluster,title
267554,5218322,2,The Eagle


In [193]:
#This function select the cluster for a user according the the user choice
def select_c():
    global l
    print('Select The Movies Id you would like to watch:')
    l=[]
    for i in range(15):
        l.append(random.randint(0,354639))
    for i in l:
        print(new_data['movie_id'][i] , new_data['title'][i],sep='--->')
    print('--------------------------------------------------------------------')
    l = int(input())
    l = new_data[new_data.movie_id==l].iloc[1,1]

def wrapper(self):
    if len(self) == 1:
        return converter(self.iloc[0])
    raise TypeError(f"cannot convert the series to {converter}")

    wrapper.__name__ = f"__{converter.__name__}__"
    return wrapper

In [194]:
# This is the main function which recommend you movies
def main():
    ans = False
    while not ans:
        select_c()
        print(new_data['title'][new_data.Cluster == int(l)].sample(n=10))
        print('--------------------------------------------------------------------')
        print('Do you like these movies(y/n)')
        abc = input()
        while ((abc =='y') or (abc == 'Y')):          
            print(new_data['title'][new_data.Cluster == int(l)].sample(n=10))
            print('--------------------------------------------------------------------')
            print('Want more!!!!(y/n)')
            abc = input()
            if ((abc =='N') or (abc == 'n')):
                ans =True

In [198]:
main()

Select The Movies Id you would like to watch:
5514030--->Shaft
2553108--->Aftermath
4443976--->Inside Man
5584598--->The Holiday
591696--->Finders Keepers
751300--->Beauty and the Beast
2279948--->The Void
528422--->The Boy Next Door
1528481--->Beauty and the Beast
86202--->Robin Hood
6527272--->Unforgettable
556686--->Oblivion
1413435--->Blitz
748733--->24
5072932--->Tell
--------------------------------------------------------------------
1413435
192155           Blitz
194051          Inside
172144       Gladiator
190711        Roadkill
223921    Sucker Punch
207003        Unbroken
200711        Identity
190951        Roadkill
217481      Date Night
189857        Whiplash
Name: title, dtype: object
--------------------------------------------------------------------
Do you like these movies(y/n)
y
188153              Inside Man
217585                    Numb
194398                  Inside
199255                  Inside
215160              Inside Man
202311    Beauty and the Beast
211

![](image0.png)